# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# csv from WeatherPy output is saved as a dataframe.
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_df.head()

,City,Latitude,Longitude,Maximum Temperature (°F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Iqaluit,63.75,-68.51,57.200,62,90,2.60,CA,2020-07-23
1,Hobart,-42.88,147.33,44.996,75,75,5.10,AU,2020-07-23
2,Yellowknife,62.46,-114.35,60.800,82,75,6.20,CA,2020-07-23
3,Punta Arenas,-53.15,-70.92,39.200,80,75,6.20,CL,2020-07-23
4,Guerrero Negro,27.98,-114.06,72.212,61,1,8.07,MX,2020-07-23


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Base map is crafted
fig = gmaps.figure()

# Location and weights are defined for ease of reading
location_heat = cities_df[["Latitude", "Longitude"]].astype(float)
weights = cities_df["Humidity (%)"]

# Heat layer is crafted
heat_layer = gmaps.heatmap_layer(location_heat,
                                 weights=weights,
                                 max_intensity=100,
                                point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Show layer
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Locations are limited by personal preference.  No clouds, low wind, and temperatures between 70 and 80 degrees Farenheit.
location_df = cities_df.loc[cities_df["Cloudiness (%)"] == 0]\
                    .loc[cities_df["Wind Speed (mph)"] < 10]\
                    .loc[cities_df["Maximum Temperature (°F)"] < 80]\
                    .loc[cities_df["Maximum Temperature (°F)"] > 70]

# Drop rows with missing data
location_df.dropna(inplace=True)

# Output the first 10 locations fitting these criteria
location_df.head(10)

,City,Latitude,Longitude,Maximum Temperature (°F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
13,Arraial do Cabo,-22.97,-42.03,71.600,83,0,4.60,BR,2020-07-23
31,Poum,-20.23,164.02,73.832,75,0,7.14,NC,2020-07-23
48,Djebeniana,35.03,10.91,78.800,61,0,1.00,TN,2020-07-23
76,Vilhena,-12.74,-60.15,75.200,53,0,3.10,BR,2020-07-23
110,Tiznit Province,29.58,-9.50,79.376,33,0,0.80,MA,2020-07-23
121,Barra do Garças,-15.89,-52.26,72.554,37,0,1.11,BR,2020-07-23
126,Saint Lawrence,36.06,14.20,77.000,61,0,2.10,MT,2020-07-23
220,Nola,40.92,14.53,75.200,78,0,0.50,IT,2020-07-23
252,Alyangula,-13.85,136.42,71.600,94,0,4.32,AU,2020-07-23
304,Castelvetrano,37.68,12.79,78.008,49,0,2.60,IT,2020-07-23


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Dataframe for hotel information is crafted.
hotel_data = pd.DataFrame({"City":location_df["City"],
                         "Country":location_df["Country"],
                        "Lat": location_df["Latitude"],
                        "Lng": location_df["Longitude"]})

# Adding column of hotel name strings
hotel_data["Hotel Name"] = ""
hotel_data.head()

,City,Country,Lat,Lng,Hotel Name
13,Arraial do Cabo,BR,-22.97,-42.03,
31,Poum,NC,-20.23,164.02,
48,Djebeniana,TN,35.03,10.91,
76,Vilhena,BR,-12.74,-60.15,
110,Tiznit Province,MA,29.58,-9.50,


In [6]:
# Hotel names are grabbed from google places and placed in hotel name column
# Base url is defined
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters that will not change each iteration
params = {"type": "lodging",
          "radius": 5000,
          "key": g_key}
         
# iterrows iterates through rows of hotel dataframe
for index, row in hotel_data.iterrows():
    
    # Add location parameter
    params["location"] = f"{row['Lat']},{row['Lng']}"
    
    # Make api calls
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_data.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        0

        
# Drops every location which does not have a hotel name, whether no value or an empty string. 
hotel_data.dropna(inplace=True)
hotel_df = hotel_data.loc[hotel_data["Hotel Name"] != ""]

# Output the first 10 remaining hotels
hotel_df.head(10)

,City,Country,Lat,Lng,Hotel Name
13,Arraial do Cabo,BR,-22.97,-42.03,Pousada Porto Praia
31,Poum,NC,-20.23,164.02,Gîte kajeon
48,Djebeniana,TN,35.03,10.91,maison belmabrouk
76,Vilhena,BR,-12.74,-60.15,Hotel Mirage
121,Barra do Garças,BR,-15.89,-52.26,Hotel Pousada Tropical
126,Saint Lawrence,MT,36.06,14.20,San Andrea Hotel
220,Nola,IT,40.92,14.53,Pamaran Hotel
252,Alyangula,AU,-13.85,136.42,Groote Eylandt Lodge
304,Castelvetrano,IT,37.68,12.79,Althea Palace Hotel
322,Sabha,LY,27.04,14.43,محطة ام الارانب -سبها مواصلات


In [7]:
# NOTE: Do not change any of the code in this cell
# Roger Roger

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer on top of heat map

# Truthfully, I don't know why running this through the following for loop works, but the markers only appear if this code runs
#    through something else, so I set up a for loop that runs precisely once, and now it works.
for x in range(1,2):
    
    # Fig is redefined here and the heat layer added.
    fig = gmaps.figure()
    fig.add_layer(heat_layer)

    # The markers layer is created with the locations and infobox using above cell outputs, then added to the earlier figure.
    markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
    fig.add_layer(markers)
    x += 1
    
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))